In [61]:
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pyautogui
import config
import win32gui

In [34]:
def open():
    options = Options()
    options.add_experimental_option("prefs", { \
    "profile.default_content_setting_values.media_stream_mic": 1, 
    "profile.default_content_setting_values.media_stream_camera": 1,
    "download_restrictions": 3
  })
    options.add_argument("--disable-extensions")
    driver = webdriver.Chrome('./chromedriver.exe', options=options)
    driver.get('https://fuhsd-org.zoom.us/signin')
    elem = driver.find_element_by_xpath('//*[@id="identifierId"]')
    elem.send_keys(config.zoom_username + '\n')
    while(True):
        try:
            elem = driver.find_element_by_xpath('//*[@id="password"]/div[1]/div/div[1]/input')
            elem.send_keys(config.zoom_password + '\n')
        except:
            time.sleep(1)
        else:
            break
    time.sleep(3)
    return driver

def prep_meeting(config, meeting):
    if config.url:
        meeting.get(config.url)
    elif config.loginID:
        meeting.get('https://zoom.us/j')
        elem = meeting.find_element_by_xpath('//*[@id="join-confno"]')
        elem.send_keys(link + '\n')
    else:
        raise ValueError()

In [3]:
def enter_meeting(meeting):
    def cancel():
        cancel = None
        while cancel is None:
            cancel = pyautogui.locateOnScreen('./assets/Cancel.png', grayscale=True, confidence=.8)
            time.sleep(1)
        pyautogui.click(cancel)
    
    cancel()
    try:
        meeting.find_element_by_xpath('//*[@id="zoom-ui-frame"]/div[2]/div/div[2]/h3[2]/a').click()
    except:
        xpath = '//*[@id="zoom-ui-frame"]/div[2]/div/div[1]/div'
        elem = WebDriverWait(meeting, 15).until(EC.presence_of_element_located((By.XPATH, xpath)))
        elem.click()
        cancel()
        meeting.find_element_by_xpath('//*[@id="zoom-ui-frame"]/div[2]/div/div[2]/h3[2]/a').click()
    meeting.find_element_by_xpath('//*[@id="video-icon"]').click()
    meeting.find_element_by_xpath('//*[@id="mic-icon"]').click()
    elem = meeting.find_element_by_xpath('//*[@id="joinBtn"]')
    elem.click()


In [4]:
def maintain_meeting(config):
    try:
        time.sleep(2)
        elem = meeting.find_element_by_xpath('//*[@id="inputpasscode"]')
        elem.send_keys(config.password + '\n')
    except:
        0
    xpath = '//*[@id="wc-footer"]/div/div[2]/button[1]/div/div/span/span'
    elem = WebDriverWait(meeting, 15).until(EC.presence_of_element_located((By.XPATH, xpath)))
    count = int(elem.get_attribute('innerHTML'))
    old = count;
    while(count + config.thresh > old):
        print('Participants:', count)
        old = count
        time.sleep(5)
        count = int(elem.get_attribute('innerHTML'))
    meeting.quit()

In [58]:
def default_webcam(config, meeting):
    if config.video_file == 0:
        return
    meeting.get('chrome://settings/content/camera')
    hwnd = win32gui.FindWindow(None, 'Settings - Camera - Google Chrome')
    win32gui.SetForegroundWindow(hwnd)

    time.sleep(1)
    loc = pyautogui.locateOnScreen('./assets/DropDown.png', grayscale=False, confidence=.8)
    pyautogui.click(loc)
    if type(config.video_file).__name__ == 'int':
        for i in range(config.video_file):
            pyautogui.press('down')
        pyautogui.press('enter')
    else:
        loc = pyautogui.locateOnScreen('./assets/OBS-camera.png')
        pyautogui.click(loc)

In [35]:
def start(config):
    meeting = open()
    default_webcam(config, meeting)
    prep_meeting(config, meeting)
    enter_meeting(meeting)
    maintain_meeting(meeting)

In [59]:
if __name__ == '__main__':
    #config.url = #insert test zoom link
    #config.password = # or zoom meeting password
    #config.loginID = # and zoom id
    config.video_file = 'ex'
    start(config)
    

ValueError: 